In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (TimeoutException, NoSuchElementException, 
                                       NoSuchWindowException, WebDriverException)
import time
import random
import pandas as pd
import os
import socket
import subprocess
import re

# 配置Edge浏览器（强化反检测版本）
options = webdriver.EdgeOptions()

# 核心反检测配置
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
options.add_argument('--disable-infobars')
options.add_argument('--start-maximized')

# 隐藏自动化开关
options.add_experimental_option('excludeSwitches', ['enable-automation', 'enable-logging'])
options.add_experimental_option('useAutomationExtension', False)

# 优化User-Agent
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.67')

# 配置用户数据目录（使用临时目录避免冲突）
user_data_path = r"C:\Temp\EdgeCrawlerData"  # 专用临时目录
if not os.path.exists(user_data_path):
    os.makedirs(user_data_path)
options.add_argument(f"user-data-dir={user_data_path}")

# 本地EdgeDriver路径配置
edgedriver_path = r'D:\edgedriver_win64\msedgedriver.exe'
if not os.path.exists(edgedriver_path):
    raise FileNotFoundError(f"未找到EdgeDriver: {edgedriver_path}")

# 目标URL - 新浪财经深A股页面
base_url = "https://vip.stock.finance.sina.com.cn/mkt/#sz_a"      

# 网络连接检查
def check_internet_connection():
    test_ips = [("8.8.8.8", 53), ("114.114.114.114", 53), ("223.5.5.5", 53)]
    for ip, port in test_ips:
        try:
            socket.create_connection((ip, port), timeout=3)
            print(f"网络连接正常（通过{ip}验证）")
            return True
        except OSError:
            continue
    print("网络连接失败，请检查网络设置")
    return False

# URL验证
def verify_url_accessible(url):
    try:
        if url.startswith(('http://', 'https://')):
            domain = url.split('/')[2]
        else:
            domain = url.split('/')[0]
        
        ip_address = socket.gethostbyname(domain)
        print(f"域名 {domain} 解析正常，IP: {ip_address}")
        
        socket.create_connection((ip_address, 80), timeout=5)
        print(f"成功与 {domain} 建立连接")
        return True
    except Exception as e:
        print(f"域名验证失败: {str(e)}")
        return False

# 随机等待
def random_sleep(min_seconds=2, max_seconds=5, human_like=True):
    if human_like:
        if random.random() < 0.1:
            sleep_time = random.uniform(max_seconds, max_seconds * 2)
        else:
            sleep_time = random.uniform(min_seconds, max_seconds)
    else:
        sleep_time = random.uniform(min_seconds, max_seconds)
    
    time.sleep(sleep_time)
    return sleep_time

# 初始化浏览器
def init_browser():
    service = Service(edgedriver_path)
    driver = webdriver.Edge(service=service, options=options)
    
    # 隐藏webdriver属性
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined,
                configurable: true
            });
            Object.defineProperty(navigator, 'plugins', {
                get: () => [1, 2, 3],
                configurable: true
            });
        """
    })
    
    driver_pid = driver.service.process.pid
    print(f"当前浏览器进程ID：{driver_pid}")
    return driver, driver_pid

# 安全关闭浏览器
def safe_close_browser(driver, pid):
    try:
        if driver:
            driver.quit()
            print("已正常关闭爬虫浏览器")
    except Exception as e:
        print(f"正常关闭浏览器失败，尝试强制关闭进程：{e}")
        try:
            subprocess.run(["taskkill", "/F", "/PID", str(pid)], 
                          stdout=subprocess.PIPE, 
                          stderr=subprocess.PIPE,
                          check=True)
            print(f"已强制关闭进程PID：{pid}")
        except Exception as ex:
            print(f"强制关闭进程失败：{ex}")

# 提取股票基本信息
def extract_stock_basic_info(driver, stock_idx):
    try:
        # 提取股票代码
        code_xpath = f"//*[@id='tbl_wrap']/table/tbody/tr[{stock_idx}]/th[1]/a"
        code_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, code_xpath))
        )
        stock_code = code_element.text.strip()
        
        # 提取股票名称
        name_xpath = f"//*[@id='tbl_wrap']/table/tbody/tr[{stock_idx}]/th[2]/a"
        name_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, name_xpath))
        )
        stock_name = name_element.text.strip()
        
        # 提取最新价
        price_xpath = f"//*[@id='tbl_wrap']/table/tbody/tr[{stock_idx}]/td[1]"
        price_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, price_xpath))
        )
        latest_price = price_element.text.strip()
        
        return {
            "股票名称": stock_name,
            "股票代码": stock_code,
            "最新价": latest_price
        }
    
    except Exception as e:
        print(f"提取基本信息出错: {e}")
        return {"股票名称": "", "股票代码": "", "最新价": ""}

# 提取股票评论（每支股票只爬前20页，包含作者提取）
def extract_all_comments(driver, stock_name, stock_code, max_comment_pages, main_window, all_stock_data):
    all_comments = []
    max_retry = 3
    retry_count = 0
    last_page = 1
    
    # 保存评论标题页的窗口句柄
    comment_list_window = None
    
    while retry_count < max_retry:
        try:
            if not driver.window_handles:
                raise NoSuchWindowException("页面已关闭")
            
            # 记录评论列表页的窗口句柄并验证不是主窗口
            comment_list_window = driver.current_window_handle  
            if comment_list_window == main_window:
                raise Exception("错误：评论列表页与主窗口句柄冲突")
            print("✅ 进入股吧评论区，已记录评论列表页窗口")
            current_page = last_page
            
            # 只爬取到max_comment_pages（20页）为止
            while current_page <= max_comment_pages:
                print(f"\n===== 开始处理{stock_name}评论第{current_page}页 =====")
                page_comments = []
                
                # 提取当前页的评论标题和作者（每页60条）
                for comment_idx in range(2, 62):
                    try:
                        # 确保我们在评论列表页（非主窗口）
                        if driver.current_window_handle != comment_list_window:  
                            print("⚠️ 意外离开评论列表页，正在切换回去...")
                            # 验证评论列表窗口是否存在
                            if comment_list_window not in driver.window_handles:
                                raise NoSuchWindowException("评论列表窗口已关闭")
                            driver.switch_to.window(comment_list_window)
                            random_sleep(2, 3)
                        
                        # 评论标题定位
                        title_xpath = f"//*[@id='blk_list_02']/table/tbody/tr[{comment_idx}]/td[3]/a"
                        title_element = WebDriverWait(driver, 10).until(  
                            EC.element_to_be_clickable((By.XPATH, title_xpath))
                        )
                        comment_title = title_element.text.strip()
                        
                        # 提取评论作者
                        author_xpath = f"//*[@id='blk_list_02']/table/tbody/tr[{comment_idx}]/td[4]/div/a"
                        try:
                            author_element = WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.XPATH, author_xpath))
                            )
                            comment_author = author_element.text.strip()
                        except:
                            comment_author = "未知作者"
                        
                        # 记录当前窗口数
                        initial_window_count = len(driver.window_handles)
                        
                        # 打开评论详情页
                        title_element.click()
                        random_sleep(2, 4)
                        
                        # 切换到新打开的窗口
                        if len(driver.window_handles) > initial_window_count:
                            # 找到新打开的窗口（排除评论列表页和主窗口）
                            new_window = None
                            for handle in driver.window_handles:
                                if handle != comment_list_window and handle != main_window:
                                    new_window = handle
                                    break
                            
                            if not new_window:
                                print("ℹ️ 未找到新打开的评论窗口")
                                continue
                            
                            driver.switch_to.window(new_window)
                            
                            # 提取完整评论内容
                            content_xpath = "//*[@id='thread_content']"
                            try:
                                content_element = WebDriverWait(driver, 15).until(
                                    EC.presence_of_element_located((By.XPATH, content_xpath))
                                )
                                comment_content = content_element.text.strip()
                                
                                page_comments.append({
                                    "页码": current_page,
                                    "评论序号": comment_idx - 1,  # 从1开始计数
                                    "标题": comment_title,
                                    "作者": comment_author,
                                    "内容": comment_content
                                })
                                print(f"✅ 提取第{comment_idx - 1}条评论（作者：{comment_author}）")
                            except TimeoutException:
                                print(f"ℹ️ 第{comment_idx - 1}条评论内容未加载")
                            
                            # 关闭当前评论窗口，回到列表页
                            driver.close()
                            # 确保切换回评论列表页
                            driver.switch_to.window(comment_list_window)
                        else:
                            print(f"ℹ️ 第{comment_idx - 1}条评论未打开新窗口")
                        
                        random_sleep(1, 3)
                        
                    except Exception as e:
                        print(f"❌ 第{comment_idx - 1}条评论处理失败: {e}")
                        # 发生错误时确保回到评论列表页
                        if driver.window_handles and comment_list_window in driver.window_handles:
                            driver.switch_to.window(comment_list_window)
                        continue
                
                if page_comments:
                    all_comments.extend(page_comments)
                    print(f"✅ 第{current_page}页共提取 {len(page_comments)} 条评论")
                
                # 点击下一页
                try:
                    # 确保我们在评论列表页
                    if driver.current_window_handle != comment_list_window:
                        print("⚠️ 翻页前不在评论列表页，正在切换回去...")
                        driver.switch_to.window(comment_list_window)
                        random_sleep(2, 3)
                    
                    next_button = WebDriverWait(driver, 15).until(
                        EC.element_to_be_clickable((By.LINK_TEXT, "下一页"))
                    )
                    
                    # 检查是否为最后一页
                    btn_class = next_button.get_attribute("class")
                    if btn_class and "disabled" in btn_class:
                        print("⚠️ 已到网站最后一页，提前停止")
                        break
                    
                    # 点击前再次确认窗口
                    if driver.current_window_handle == comment_list_window:
                        next_button.click()
                        print(f"✅ 已点击“下一页”按钮，前往第{current_page + 1}页")
                        random_sleep(3, 5)
                        
                        # 验证页面跳转成功
                        try:
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.XPATH, f"//a[text()='{current_page + 1}']"))
                            )
                            current_page += 1
                            last_page = current_page
                        except TimeoutException:
                            print("⚠️ 页码更新验证失败，可能未成功跳转")
                            driver.refresh()
                            random_sleep(3, 5)
                    else:
                        print("⚠️ 不在评论列表页，无法点击下一页")
                        break
                
                except TimeoutException:
                    print("❌ 未找到“下一页”按钮，已到网站最后一页")
                    break
                except Exception as e:
                    print(f"❌ 点击下一页出错: {e}")
                    break
            
            if all_comments:
                break
            else:
                retry_count += 1
                print(f"⚠️ 未提取到评论，第{retry_count}次重试当前股票...")
                time.sleep(5)
        
        except Exception as e:
            retry_count += 1
            print(f"❌ 评论提取出错，第{retry_count}次重试: {e}")
            
            if "session" in str(e).lower() or "window" in str(e).lower():
                print(f"⚠️ 会话失效/页面关闭，尝试重新初始化浏览器...")
                driver.quit()
                driver, _ = init_browser()
                driver.get(base_url)
                time.sleep(5)
                print("请重新完成登录/验证（30秒内）...")
                time.sleep(30)
            else:
                # 发生错误时尝试回到评论列表页
                if driver.window_handles and comment_list_window in driver.window_handles:
                    driver.switch_to.window(comment_list_window)
                driver.refresh()
                time.sleep(5)
    
    print(f"\n===== {stock_name}评论提取完成，共提取 {len(all_comments)} 条评论 =====")
    return all_comments

# 保存数据到Excel（包含作者字段）
def save_to_excel(all_stock_data, batch_size=1000, filename="新浪财经股票数据.xlsx"):
    try:
        excel_data = []  
        for i, stock in enumerate(all_stock_data):
            basic_info = stock["基本信息"]
            comments = stock["评论"]
            
            for comment in comments:
                row = {
                    "股票代码": basic_info.get("股票代码", ""),
                    "股票名称": basic_info.get("股票名称", ""),
                    "最新价": basic_info.get("最新价", ""),
                    "评论页码": comment.get("页码", ""),
                    "评论序号": comment.get("评论序号", ""),
                    "评论作者": comment.get("作者", ""),
                    "评论标题": comment.get("标题", ""),
                    "评论内容": comment.get("内容", "")
                }
                excel_data.append(row)
            
            if (i + 1) % batch_size == 0:
                df = pd.DataFrame(excel_data)
                df.to_excel(f"{filename}_part{i//batch_size + 1}.xlsx", index=False)  
                print(f"已保存第{i//batch_size + 1}批数据（共{len(excel_data)}条）")
                excel_data = []
        
        if excel_data:
            df = pd.DataFrame(excel_data)
            if len(all_stock_data) > batch_size:
                df.to_excel(f"{filename}_part{len(all_stock_data)//batch_size + 1}.xlsx", index=False)
            else:
                df.to_excel(filename, index=False)
            print(f"\n成功保存 {len(excel_data)} 条评论数据到 {filename}")
        
    except Exception as e:
        print(f"保存Excel失败: {e}")

# 主函数（只爬取1只股票，每支股票爬前20页评论）
def main():
    all_stock_data = []
    driver = None
    driver_pid = None
    main_window = None  # 主页面窗口句柄（股票列表页）
    
    try:
        # 检查网络和URL可用性
        if not check_internet_connection():
            raise ConnectionError("网络连接失败")  
        if not verify_url_accessible(base_url):
            raise ConnectionError("目标URL无法访问")
        
        # 初始化浏览器
        driver, driver_pid = init_browser()
        driver.get(base_url)
        main_window = driver.current_window_handle  # 记录主窗口句柄
        print(f"✅ 已记录主页面窗口句柄: {main_window}")
        random_sleep(5, 8)
        
        # 处理验证
        print("请检查浏览器窗口，如有滑动验证请手动完成，完成后按Enter继续...")
        input("确认验证完成后按Enter继续...")
        random_sleep(3, 5)
        
        # 爬取参数
        total_stock_pages = 1  # 只爬第一页
        stocks_per_page = 40
        max_stock_retries = 3  
        max_comment_pages = 20  # 每只股票只爬前20页评论
        target_stock_index = 1  # 只爬取第1支股票（可修改为1-40之间的数字）
        
        # 爬取逻辑
        for stock_page in range(1, total_stock_pages + 1):
            print(f"\n===== 正在爬取股票列表第{stock_page}页 =====")
            
            try:
                # 确保在主页面
                if driver.current_window_handle != main_window:
                    print("⚠️ 不在主页面，正在切换回去...")
                    if main_window not in driver.window_handles:
                        raise NoSuchWindowException("主页面窗口已关闭！")
                    driver.switch_to.window(main_window)
                    random_sleep(2, 3)
                
                if stock_page > 1:
                    try:
                        page_button = WebDriverWait(driver, 15).until(
                            EC.element_to_be_clickable((By.LINK_TEXT, str(stock_page)))
                        )
                        page_button.click()
                    except:
                        print(f"⚠️ 直接定位第{stock_page}页失败，尝试通过下一页翻页")
                        next_page_btn = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.LINK_TEXT, "下一页"))
                        )
                        next_page_btn.click()
                    
                    random_sleep(5, 8)
                    
                    # 验证是否跳转到目标页
                    try:
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, f"//a[text()='{stock_page}' and contains(@class, 'cur')]"))  
                        )
                        print(f"✅ 已成功跳转至第{stock_page}页")
                    except:
                        print(f"⚠️ 第{stock_page}页跳转失败，跳过当前页")
                        continue
                
                # 只爬取1只股票（修改target_stock_index可指定具体股票）
                stock_idx = target_stock_index
                stock_retry = 0
                stock_success = False
                comment_window = None  # 评论列表页窗口句柄
                
                while stock_retry <= max_stock_retries and not stock_success:
                    try:
                        print(f"\n----- 正在爬取第{stock_page}页第{stock_idx}支股票（第{stock_retry + 1}次尝试） -----")
                        
                        # 检查会话是否有效并确保在主页面
                        if not driver.session_id or not driver.window_handles:
                            print("⚠️ 会话失效，重新初始化浏览器...")
                            safe_close_browser(driver, driver_pid)
                            driver, driver_pid = init_browser()
                            driver.get(base_url)
                            main_window = driver.current_window_handle
                            random_sleep(5)
                            print("请重新完成登录/验证（30秒）...")
                            random_sleep(30)
                            driver.get(f"{base_url}#sz_a&page={stock_page}")
                            random_sleep(5, 8)
                        else:
                            # 确保在主页面
                            if driver.current_window_handle != main_window:
                                print("⚠️ 处理股票前不在主页面，正在切换回去...")
                                if main_window not in driver.window_handles:
                                    raise NoSuchWindowException("主页面窗口已关闭，无法继续")
                                driver.switch_to.window(main_window)
                                random_sleep(2, 3)
                        
                        # 提取股票基本信息
                        stock_basic = extract_stock_basic_info(driver, stock_idx)
                        if not stock_basic["股票名称"]:
                            print(f"⚠️ 未获取到股票信息，跳过第{stock_idx}支股票")
                            break
                        
                        print(f"正在处理股票: {stock_basic['股票名称']}（{stock_basic['股票代码']}）")
                        
                        # 点击股吧按钮进入评论区
                        stock_link = WebDriverWait(driver, 20).until(
                            EC.element_to_be_clickable((By.XPATH,
                                f"//*[@id='tbl_wrap']/table/tbody/tr[{stock_idx}]/th[3]/a"))
                        )
                        
                        # 记录当前窗口数
                        initial_window_count = len(driver.window_handles)
                        stock_link.click()
                        random_sleep(5, 8)
                        
                        # 切换到新窗口（评论列表页）
                        if len(driver.window_handles) > initial_window_count:
                            # 找到评论列表窗口（排除主窗口）
                            comment_window = None
                            for handle in driver.window_handles:
                                if handle != main_window:
                                    comment_window = handle
                                    break
                            
                            if not comment_window:
                                raise Exception("未找到评论列表窗口")
                                
                            driver.switch_to.window(comment_window)
                            print(f"✅ 已切换到评论列表页，窗口句柄: {comment_window}")
                        else:
                            print("⚠️ 未打开新窗口，使用当前窗口作为评论列表页")
                            comment_window = driver.current_window_handle
                        
                        # 提取评论
                        stock_comments = extract_all_comments(
                            driver,   
                            stock_basic["股票名称"], 
                            stock_basic["股票代码"],
                            max_comment_pages,  
                            main_window,
                            all_stock_data
                        )
                        print(f"共提取到{len(stock_comments)}条评论（当前股票）")
                        
                        # 保存数据
                        stock_data = {
                            "基本信息": stock_basic,
                            "评论": stock_comments
                        }
                        all_stock_data.append(stock_data)
                        stock_success = True
                        
                        # 关闭评论窗口，回到股票列表主页面
                        if comment_window and comment_window != main_window:  
                            if comment_window in driver.window_handles:
                                driver.switch_to.window(comment_window)
                                driver.close()  
                                print(f"✅ 已关闭评论列表窗口（{comment_window}）")
                            
                            # 确保切换回主窗口
                            if main_window in driver.window_handles:
                                driver.switch_to.window(main_window)
                                print("✅ 已回到主页面")
                            else:
                                raise NoSuchWindowException("主页面窗口丢失！")
                        
                        random_sleep(3, 5)
                        
                    except Exception as e:
                        stock_retry += 1
                        print(f"处理第{stock_page}页第{stock_idx}支股票时出错（第{stock_retry}次）: {e}")
                        
                        try:
                            # 确保关闭评论窗口并回到主页面
                            if driver and driver.window_handles:
                                # 关闭评论窗口（如果存在且不是主窗口）
                                if comment_window and comment_window != main_window and comment_window in driver.window_handles:
                                    driver.switch_to.window(comment_window)
                                    driver.close()
                                
                                # 切换回主页面（如果存在）
                                if main_window in driver.window_handles:
                                    driver.switch_to.window(main_window)
                            random_sleep(3)
                        except:
                            pass
                        
                        if stock_retry > max_stock_retries:
                            print(f"⚠️ 已达最大重试次数，跳过当前股票")
        
            except Exception as e:
                print(f"爬取第{stock_page}页出错: {e}")
                # 出错时确保回到主页面
                if driver and main_window in driver.window_handles:
                    driver.switch_to.window(main_window)
                driver.refresh()
                random_sleep(5)  
                continue
        
        # 最终保存所有数据
        save_to_excel(all_stock_data)
        print("所有爬取任务已完成")
        
    # 捕获内核中断事件
    except KeyboardInterrupt:
        print("\n⚠️ 检测到内核中断，正在强制保存数据...")
        if all_stock_data:
            save_to_excel(all_stock_data, filename="新浪财经股票数据_中断保存.xlsx")
            print(f"已保存{len(all_stock_data)}支股票的信息到Excel")
        else:
            print("当前没有可保存的数据")
    # 捕获其他异常
    except Exception as e:
        print(f"爬取过程出错：{e}")
        if all_stock_data:  
            save_to_excel(all_stock_data, filename="新浪财经股票数据_错误保存.xlsx")
            print("已保存部分爬取数据")
    finally:
        safe_close_browser(driver, driver_pid)


if __name__ == "__main__":
    main()

网络连接正常（通过8.8.8.8验证）
域名 vip.stock.finance.sina.com.cn 解析正常，IP: 106.63.15.52
成功与 vip.stock.finance.sina.com.cn 建立连接
当前浏览器进程ID：4856
✅ 已记录主页面窗口句柄: CC2A0C29486E17E11F742B4DCF73FB35
请检查浏览器窗口，如有滑动验证请手动完成，完成后按Enter继续...


确认验证完成后按Enter继续... 



===== 正在爬取股票列表第1页 =====

----- 正在爬取第1页第1支股票（第1次尝试） -----
正在处理股票: 平安银行（sz000001）
✅ 已切换到评论列表页，窗口句柄: C028AF023D6215C639189B2DC0DA8204
✅ 进入股吧评论区，已记录评论列表页窗口

===== 开始处理平安银行评论第1页 =====
✅ 提取第1条评论（作者：镇荒apAll小贴士）
✅ 提取第2条评论（作者：小_中_暂不可见）
✅ 提取第3条评论（作者：波比繁华）
✅ 提取第4条评论（作者：你是我的韦依）
✅ 提取第5条评论（作者：风华绝代的Toby）
✅ 提取第6条评论（作者：魏世thenorth）
✅ 提取第7条评论（作者：Ko的天马）
✅ 提取第8条评论（作者：勇敢的心198168）
✅ 提取第9条评论（作者：斗斗working）
✅ 提取第10条评论（作者：用户_7540927813）
✅ 提取第11条评论（作者：问鼎资本-张佳男）
✅ 提取第12条评论（作者：镇荒apAll小贴士）
✅ 提取第13条评论（作者：镇荒apAll小贴士）
✅ 提取第14条评论（作者：魏世thenorth）
✅ 提取第15条评论（作者：你是我的韦依）
✅ 提取第16条评论（作者：江西老表说股票）
✅ 提取第17条评论（作者：魏世thenorth）
✅ 提取第18条评论（作者：新浪财经客户端）
✅ 提取第19条评论（作者：镇荒apAll小贴士）
✅ 提取第20条评论（作者：书山跑步背单词）
✅ 提取第21条评论（作者：王斌3490755）
✅ 提取第22条评论（作者：寂小童_92）
✅ 提取第23条评论（作者：念念的小火炉）
✅ 提取第24条评论（作者：打鱼种稻）
✅ 提取第25条评论（作者：镇荒apAll小贴士）
✅ 提取第26条评论（作者：二马由之）
✅ 提取第27条评论（作者：镇荒apAll小贴士）
✅ 提取第28条评论（作者：小_中_暂不可见）
✅ 提取第29条评论（作者：镇荒apAll小贴士）
✅ 提取第30条评论（作者：江西老表说股票）
✅ 提取第31条评论（作者：投基大圣哈哈）
✅ 提取第32条评论（作者：镇荒apAll小贴士）
✅ 提取第33条评论（作者：卜平安）
✅ 提取第34条评论（作者：镇荒apAll小贴士）
✅ 提取第35条评论（作者